<a href="https://colab.research.google.com/github/sahaanirbannew/birdname-ner-nlp-demo/blob/main/Program_to_annotate_bird_tweets_and_train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs, Imports, Function Definitions


In [2]:
!pip install tweet-preprocessor
!pip install ekphrasis
!pip install Levenshtein  
!python -m spacy download en_core_web_sm 
!pip install spacy[transformers]
!python -m spacy download en_core_web_lg 
!python -m spacy download en_core_web_trf 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.6 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 4.1 MB/s 
     |████████████████████████████████| 1.6 MB 49.3 MB/s 
     |████████████████████████████████| 104 kB 70.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 4.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.6 MB/s 
     

In [3]:
from google.colab import drive
import pickle 
from Levenshtein import distance as levenshtein_distance
import re
import tweepy
import preprocessor as p
p.set_options(p.OPT.EMOJI, p.OPT.MENTION, p.OPT.URL, p.OPT.SMILEY, p.OPT.NUMBER, p.OPT.HASHTAG)
from ekphrasis.classes.segmenter import Segmenter
seg_tw = Segmenter(corpus="english")
import requests
import csv
import datetime 
from datetime import datetime 
import os, sys 
import pandas as pd  
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
from spacy.util import filter_spans
import shutil
from zipfile import ZipFile
import tensorflow as tf

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading english - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_1grams.txt
Reading english - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_2grams.txt


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [4]:
spelling_corrections = {}
spelling_corrections["grey"] = "gray" 
spelling_corrections["pegion"] = "pigeon" 
spelling_corrections["brested"] = "breasted" 
spelling_corrections["serpant"] = "serpent" 
spelling_corrections["avedavat"] = "avadavat" 
spelling_corrections["open billed stork"] = "asian openbill" 
spelling_corrections["secretary bird"] = "Secretarybird" 
spelling_corrections["dollar bird"] = "dollarbird"
spelling_corrections["silver bill"] = "silverbill"

def create_twitter_app_obj():
  consumer_key = "iPaIdR8GRI59yTJMs0Es0dIBN"
  consumer_secret = "pLadg3UaLeK3yKDujRMChRN3p8hUDBOjBsuOBy8j8ERr4zz1vs"
  access_token = "39085479-AabHt6bmFSbClDfUZuHjModYPAxVlOxHeMA79UyVt"
  access_token_secret = "3IqXDISfqg14wzMNNn2AX4KYG9Wfkltt21QxKasE4YNnG"
  # Creating the authentication object
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  # Setting your access token and secret
  auth.set_access_token(access_token, access_token_secret)
  # Creating the API object while passing in auth information

  try:
    api = tweepy.API(auth) 
    return api
  except:
    print("Error: Error making the Twitter Object.")
    return 0

twitter = create_twitter_app_obj() 

def connect_to_google_drive():
  drive.mount("/content/drive", force_remount=False)
connect_to_google_drive()

def get_google_drive_folder_path():
  path = "/content/drive/My Drive/IndiAves/" 
  return path 
google_drive_path= get_google_drive_folder_path() 

def get_eBird_commonNames_data(path):
  file = open(path+"bird_dict_comName",'rb')
  try:
    eBird_commonNames_data = pickle.load(file)
    return eBird_commonNames_data 
  except Exception as e:
    print(str(e))
    return 0
ebirds = get_eBird_commonNames_data(google_drive_path)

def load_all_birds_list(path):
  file = open(path+"bird_list_df",'rb')
  bird_list_df = pickle.load(file)
  try: 
    return bird_list_df
  except:
    print("Error: No bird list found.")
    return 0
wikibirds = load_all_birds_list(google_drive_path) 

def basic_preprocess(tweet, spelling_corrections):
  tweet = tweet.lower()
  tweet = p.clean(tweet)
  tweet = tweet.replace("\n"," ")  
  tweet = tweet.replace("\\n"," ") 
  if tweet[:2] == "b'": tweet = tweet[1:] 
  tweet = tweet.replace("'","")
  tweet = tweet.replace("#","")
  tweet = re.sub(r'[^\w\s]', ' ', tweet)
  tweet = re.sub(r' x..', '', tweet)
  tweet = re.sub(r' +', ' ', tweet) 
  tweet = tweet.replace("x9c","")
  tweet = tweet.strip()
  for key in spelling_corrections: 
    if tweet.find(key)>-1: 
      tweet = tweet.replace(key,spelling_corrections[key])
  return tweet 

def add_words_to_bird_vocab(bird_name, list_, spelling_corrections): 
  words = basic_preprocess(bird_name,spelling_corrections).split(" ")
  for word in words:
    if word not in list_:
      list_.append(word)
  return list_

def add_birdnames_to_list(bird_name, list_, spelling_corrections): 
  bird_name__ = basic_preprocess(bird_name, spelling_corrections) 
  if bird_name__ not in list_: 
    list_.append(bird_name__) 
  return list_

all_birds_list = [] 
birdnames_words = [] 
birdnames = wikibirds["bird_name"].unique().tolist() 
for speciesTag in ebirds:
  all_birds_list = add_birdnames_to_list(ebirds[speciesTag], all_birds_list, spelling_corrections)
  birdnames_words = add_words_to_bird_vocab(ebirds[speciesTag], birdnames_words, spelling_corrections) 
for birdname in birdnames: 
  all_birds_list = add_birdnames_to_list(birdname, all_birds_list, spelling_corrections)
  birdnames_words = add_words_to_bird_vocab(birdname, birdnames_words, spelling_corrections) 

def return_alt_word(word_,birdnames_words): 
  min_distance = 1000
  if word_ not in birdnames_words: 
    for word in birdnames_words: 
      dist_ = levenshtein_distance(word_,word)
      if dist_ < min_distance: 
        min_distance = dist_
        word__ = word 
  else: 
    return word_  
  return word__

def return_alt_bird(word_,all_birds_list): 
  min_distance = 1000
  if word_ not in all_birds_list: 
    for word in all_birds_list: 
      dist_ = levenshtein_distance(word_,word)
      if dist_ < min_distance: 
        min_distance = dist_
        word__ = word 
  else: 
    return word_  
  return word__

def try_removing_hashtags(text,all_birds_list,birdnames_words):
  status = False 
  #print(text)
  hashtags = re.findall(r"#(\w+)", text) 
  for hashtag in hashtags:
    segmented_ = seg_tw.segment(hashtag) 
    words = segmented_.split(" ") #ekhane spelling mistake thakbe na. 
    #for word in words: #in case there is a spelling mistake. #works only when there is a spelling mistake
      #segmented_ = segmented_.replace(word, return_alt_word(word,birdnames_words))
    for bird in all_birds_list: #not an exact match, because people always do not write the full name.
      if bird.find(segmented_) > -1: 
        text = text.replace("#"+hashtag,segmented_)
        status = True
  return text

def get_bird_names(tweet, birdnames_words):
  api_url = "https://bird-name-ner-nlp.herokuapp.com/ner?sent="+tweet
  response = requests.get(api_url).json()
  #print("NER",response['bird-ner'])
  #print("WIKI",response['bird-wiki'])
  #print("EBIRD",response['bird-ebird'])
  bird_list_= [] 
  for bird in response['bird-wiki']: 
    if bird not in bird_list_: 
      bird_list_.append(bird) 
  for bird in response['bird-ebird']: 
    if bird not in bird_list_: 
      bird_list_.append(bird) 
  
  for bird in response['bird-ner']:
    status_ = False 
    if bird not in bird_list_: 
      #check for spelling errors.
      for word in bird.split(" "):
        bird = bird.replace(word, return_alt_word(word,birdnames_words)) 

      if len(bird)>0:
        for bird_ in bird_list_:
          if bird_.find(bird) > -1: #if it is found, then no action.  
            status_ = True #found 
            break
        if status_ == False:
          bird_list_.append(bird) 
  
  return bird_list_

def get_birds_given_text(tweet,all_birds_list, birdnames_words,spelling_corrections):
  tweet = try_removing_hashtags(tweet,all_birds_list, birdnames_words) 
  tweet = basic_preprocess(tweet, spelling_corrections) 
  print(tweet)  #needed to confirm birds.
  bird_list = get_bird_names(tweet, birdnames_words) 
  return bird_list 

Mounted at /content/drive


In [9]:
def get_since_id(path,search_word):
  file = open(path+"since_id",'rb')
  try:
    since_id_data = pickle.load(file)
  except Exception as e:
    print(str(e))
    return 0
  try: 
    since_id = since_id_data[search_word]
    return since_id
  except:
    print("Info: This looks like a new search.")
    return 0


def what_search_term():
  search_term = "indiAves"        ## CHANGE THIS.  
  return search_term

def load_training_data(path):
  try: 
    file = open(path+"training_data",'rb')
    training_data = pickle.load(file)
    return training_data
  except:
    print("Error: No training data found.")
    return 0

def import_best_model(google_drive_folder_path):
  shutil.copyfile(google_drive_folder_path+"model-best.zip", "/content/model-best.zip")

def extract_model_best_from_archive():
  file_name = "/content/model-best.zip"
  with ZipFile(file_name, 'r') as zip:
    zip.extractall("/content/model-best")

def load_best_model_from_archive(google_drive_folder_path):
  import_best_model(google_drive_folder_path)
  extract_model_best_from_archive()
  nlp_ner = spacy.load("model-best") 
  return True, nlp_ner

def add_to_database(api, search_word, ext_path, since_id):
  new_search = "#" +search_word + " -filter:retweets" 
  count = 0
  csvFile = open(ext_path+search_word+'.csv', 'a')
  csvFile2 = open('/content/temp_.csv', 'w') 

  csvWriter = csv.writer(csvFile)
  csvWriter2 = csv.writer(csvFile2)

  for tweet in tweepy.Cursor(api.search,q=new_search,count=100,
                            lang="en",
                            since_id=since_id, tweet_mode="extended").items(): 
    if tweet.id > since_id:
      since_id = tweet.id
    
    media_url=""
    if tweet.entities.get('media', []): media_url = tweet.entities.get('media', [])[0]['media_url']

    hashtags = []
    for tag in tweet.entities["hashtags"]:
      hashtags.append(tag["text"]) 
    
    csvWriter.writerow([tweet.created_at, tweet.id, tweet.user.screen_name.encode('utf-8'), tweet.user.location.encode('utf-8'), tweet.full_text.encode('utf-8'), media_url, hashtags])
    csvWriter2.writerow([tweet.created_at, tweet.id, tweet.user.screen_name.encode('utf-8'), tweet.user.location.encode('utf-8'), tweet.full_text.encode('utf-8'), media_url, hashtags])
    count +=1
  print(datetime.now(),count, "tweets retrieved.")
  return since_id 

def get_training_data(new_tweets,all_birds_list, birdnames_words, spelling_corrections, training_data): 
  for tweet in new_tweets: 
    bird_list = get_birds_given_text(tweet,all_birds_list, birdnames_words, spelling_corrections)
    print(bird_list)
    confirmation = input("Is the bird list identified, complete? y|n : ")
    if confirmation.lower() in ["yes", "y", "ja", "ya"]:
      birds = bird_list 
    else: 
      birds = input("Please confirm all birds: ")
      bird_list = birds.split(",")

    if birds != "-" and len(bird_list)>0: 
      for bird in bird_list:
        bird = bird.strip() 
        training_data["annotations"].append({'text': tweet, 'entities': [(tweet.find(bird), tweet.find(bird)+len(bird), 'BIRDNAME')]})
  return training_data

def export_training_data(training_data,path):
  filepath = open(path+"training_data",'wb') 
  pickle.dump(training_data, filepath)                     
  filepath.close() 
  print(len(training_data["annotations"]), "records in training data.")
  print("training data is saved.") 

def update_since_id(since_id, path,search_word):
  filepath = open(path+"since_id",'rb')
  since_id_data = pickle.load(filepath)
  since_id_data[search_word]=since_id
  filepath = open(path+"since_id",'wb') 
  pickle.dump(since_id_data, filepath)                     
  filepath.close()

def is_GPU_on():
  if tf.test.gpu_device_name() == "": 
    return False
  return True

def create_doc_bin(training_data,nlp):
  from spacy.tokens import DocBin
  from tqdm import tqdm
  from spacy.util import filter_spans
  doc_bin = DocBin()
  for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        #if span is None:
        #    print("Skipping entity")
        #else:
        #    ents.append(span)
        if span is not None:
          ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)
  doc_bin.to_disk("training_data.spacy") # save the docbin object

def import_base_config_file(google_drive_folder_path):
  shutil.copyfile(google_drive_folder_path+"base_config.cfg", "/content/base_config.cfg")

def archive_best_model_in_google_drive(google_folder_path):
  shutil.make_archive(google_folder_path+"model-best", 'zip', "/content/model-best")

def train_model_():
  !python -m spacy init fill-config base_config.cfg config.cfg
  !python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0


def train_model(training_data,google_drive_path): 
  nlp = spacy.blank("en")
  create_doc_bin(training_data,nlp)
  import_base_config_file(google_drive_path)
  train_model_()
  archive_best_model_in_google_drive(google_drive_path) 

# Execute this once:

In [6]:
search_word = what_search_term()                              #gets the search term = IndiAves
training_data = load_training_data(google_drive_path)

# Annotates Data, Creates Training Data

In [7]:
since_id = get_since_id(google_drive_path,search_word)        #Gets the "since_id" to fetch newer tweets. 
since_id = add_to_database(twitter,search_word, google_drive_path, since_id)  #downloads latest tweets to database, updates since_id
new_tweets_df = pd.read_csv("/content/temp_.csv", names=["created_at", "tweet_id", "user", "location", "tweet", "media_url", "hashtags"])
new_tweets = new_tweets_df["tweet"].tolist() #makes a list of the new tweets. 
training_data = get_training_data(new_tweets,all_birds_list, birdnames_words, spelling_corrections, training_data) #gets updated training data 
export_training_data(training_data,google_drive_path) #very very important.
update_since_id(since_id,google_drive_path,search_word)

2022-09-22 05:05:19.260930 0 tweets retrieved.
4480 records in training data.
training data is saved.


# Trains the Model

In [10]:
is_GPU_on_colab = is_GPU_on()
print("is_GPU_on",is_GPU_on_colab)
if is_GPU_on_colab == True: 
  train_model(training_data,google_drive_path) 

is_GPU_on True


100%|██████████| 4480/4480 [00:01<00:00, 4053.12it/s]


✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-09-22 05:08:19,977] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-09-22 05:08:19,987] [INFO] Pipeline: ['tok2vec', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'ner']
[2022-09-22 05:08:19,991] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-09-22 05:08:19,992] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
[2022-09-22 05:08:32,399] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
INFO:spacy:Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ne